In [7]:
#Importing packages
#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process
import re
import os
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
import collections
import gensim
from gensim.summarization.summarizer import summarize

C:\anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Patterns

In [74]:
#S_text =  "plaintiff motion for summary judgment is deni consideration, defend motion for summary judgment (dkt.45) is grant and deni"

#Pattern

#1. PARTIAL FOR BOTH
def text(S_text):
    A_1 = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdeni\b.*\bdefend\b.*\bgrant\b.*\bdeni', S_text)
    # Plaintiff grant deni defend grant deni
    A_2 = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdeni\b.*\bdefend\b.*\bdeni\b.*\bgrant', S_text)
    # plaintiff grant deni defend deni grant
    A_3= re.search(r'\bplaintiff\b.*\bdeni\b.*\bgrant\b.*\bdefend\b.*\bgrant\b.*\bdeni', S_text)
    # # Plaintiff deni grant defend grant deni
    A_4 = re.search(r'\bplaintiff\b.*\bbdeni\b.*\bgrant\b.*\bdefend\b.*\bdeni\b.*\bgrant', S_text)
    # Plaintiff deni grant defend deni grant
    A_4_1 = re.search(r'\bplaintiff\b.*\bbdefend\b.*\bdeni\b.*\bgrant', S_text)
    A_4_2 = re.search(r'\bplaintiff\b.*\bbdefend\b.*\bgrant\b.*\bdeni', S_text)
    
    A_1_d = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdeni\b.*\bdefend\b.*\bgrant\b.*\bdeni', S_text)
    # Plaintiff grant deni defend grant deni
    A_2_d = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdeni\b.*\bdefend\b.*\bdeni\b.*\bgrant', S_text)
    # plaintiff grant deni defend deni grant
    A_3_d= re.search(r'\bplaintiff\b.*\bdeni\b.*\bgrant\b.*\bdefend\b.*\bgrant\b.*\bdeni', S_text)
    # # Plaintiff deni grant defend grant deni
    A_4_d = re.search(r'\bplaintiff\b.*\bbdeni\b.*\bgrant\b.*\bdefend\b.*\bdeni\b.*\bgrant', S_text)
    # Plaintiff deni grant defend deni grant
    A_4_1_d = re.search(r'\bplaintiff\b.*\bbdefend\b.*\bdeni\b.*\bgrant', S_text)
    A_4_2_d = re.search(r'\bplaintiff\b.*\bbdefend\b.*\bgrant\b.*\bdeni', S_text)
    
    #Starting keywords
    A_5_1_d =  re.search(r'\bdeni.*\bdefend.*\bplaintiff', S_text)
    A_5_2_d =  re.search(r'\bgrant.*\bdefend.*\bplaintiff', S_text)
    A_5_1_p =  re.search(r'\bdefend.*\bgrant.*\bplaintiff', S_text)
    A_5_2_p =  re.search(r'\bdefend.*\bdeni.*\bplaintiff', S_text)
    A_5 = A_5_1_d and A_5_2_d and A_5_1_p and A_5_2_p
    
# plaintiff grant deni defend deni grant
    A_Part_for_both = A_1 or A_2 or A_3 or A_4 or A_4_1 or A_4_2 or  A_1_d or A_2_d or A_3_d or A_4_d or A_4_1_d or A_4_2_d or A_5

######################################################################
# Partial for one single for other
    A_5 = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdefend\b.*\bgrant\b.*\bdeni', S_text)
    A_6 = re.search(r'\bplaintiff\b.*\grant\b.*\bdefend\b.*\bdeni\b.*\bgrant', S_text)
    A_Plaintiffgrant_defendPartial = A_5 or A_6 or (A_5_2_p and A_5_1_d and A_5_2_d)

    A_5_d = re.search(r'\bdefend\b.*\bgrant\b.*\bplaintiff\b.*\bgrant\b.*\bdeni', S_text)
    A_6_d = re.search(r'\bdefend\b.*\grant\b.*\bplaintiff\b.*\bdeni\b.*\bgrant', S_text)
    A_Defendgrant_PlaintiffPartial = A_5_d or A_6_d or (A_5_1_p and A_5_2_p and A_5_2_d)
    
    A_7 = re.search(r'\bplaintiff\b.*\bdeni\b.*\bdefend\b.*\bgrant\b.*\bdeni', S_text)
    A_8 = re.search(r'\bplaintiff\b.*\bdeni\b. \bdefend\b.*\bgrant\b.*\bgrant', S_text)
    A_Plaintiffdeni_defendPartial = A_7 or A_8 or (A_5_1_p and A_5_1_d and A_5_2_d)

    A_7_d = re.search(r'\bdefend\b.*\bdeni\b.*\bplaintiff\b.*\bgrant\b.*\bdeni', S_text)
    A_8_d = re.search(r'\bdefend\b.*\bdeni\b. \bplaintiff\b.*\bgrant\b.*\bgrant', S_text)
    A_defendfdeni_PlaintiffPartial = A_7 or A_8 or (A_5_1_p and A_5_2_p and A_5_1_d)
    
    A_9 = re.search(r'\bplaintiff\b.*\bdeni\b.*\bgrant\b.*\bdefend\b.*\bgrant', S_text)
    A_10 = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdeni\b.*\bdefend\b.*\bdeni', S_text)
    A_11 = re.search(r'\bplaintiff\b.*\bdeni\b.*\bgrant\b.*\bdefend\b.*\bgrant', S_text)
    A_12 = re.search(r'\bplaintiff\b.*\bdeni\b.*\bgrant\b.*\bdefend\b.*\bdeni', S_text)
    A_PlaintiffPartial_defendDeni = A_10 or A_12
    A_PlaintiffPartial_defendGrant = A_9 or A_11

    
    A_9_d = re.search(r'\bdefend\b.*\bdeni\b.*\bgrant\b.*\bplaintiff\b.*\bgrant', S_text)
    A_10_d = re.search(r'\bdefend\b.*\bgrant\b.*\bdeni\b.*\bplaintiff\b.*\bdeni', S_text)
    A_11_d = re.search(r'\bdefend\b.*\bgrant\b.*\bdeni\b.*\bplaintiff\b.*\bgrant', S_text)
    A_12_d = re.search(r'\bdefend\b.*\bdeni\b.*\bgrant\b.*\bplaintiff\b.*\bdeni', S_text)
    A_DefendfPartial_PlaintiffDeni = A_10_d or A_12_d
    A_DefendPartial_PlaintiffGrant = A_9_d or A_11_d
    
#########################################################################
#Single for one single for other
    A_PlaintiffGrant_defendGrant = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdefend\b.*\bgrant', S_text)
    A_PlaintiffGrant_defendDeni = re.search(r'\bplaintiff\b.*\bgrant\b.*\bdefend\b.*\bdeni', S_text)
    A_PlaintiffDeni_defendGrant = re.search(r'\bplaintiff\b.*\bdeni\b.*\bdefend\b.*\bgrant', S_text)
    A_PlaintiffDeni_defendDeni = re.search(r'\bplaintiff\b.*\bdeni\b.*\bdefend\b.*\bdeni', S_text)

    A_DefendGrant_PlaintiffGrant = re.search(r'\bdefend\b.*\bgrant\b.*\bplaintiff\b.*\bgrant', S_text)
    A_DefendGrant_PlaintiffDeni = re.search(r'\bdefend\b.*\bgrant\b.*\bplaintiff\b.*\bdeni', S_text)
    A_DefendDeni_PlaintiffGrant = re.search(r'\bdefend\b.*\bdeni\b.*\bplaintiff\b.*\bgrant', S_text)
    A_DefendDeni_PlaintiffDeni = re.search(r'\bdefend\b.*\bdeni\b.*\bplaintiff\b.*\bdeni', S_text)
    
########################################################################
#Single parti judgment
    A_PlaintiffPartial1 = re.search(r'\bplaintiff\b.*\bmotion\b.*\bgrant\b.*\bdeni', S_text)
    A_PlaintiffPartial2 = re.search(r'\bplaintiff\b.*\bmotion\b.*\bdeni\b.*\bgrant', S_text)
    A_PlaintiffPartial = A_PlaintiffPartial1 or A_PlaintiffPartial2
    A_defendPartial1 = re.search(r'\bdefend\b.*\bmotion\b.*\bgrant\b.*\bdeni', S_text) 
    A_defendPartial2 =  re.search(r'\bdefend\b.*\bmotion\b.*\bdeni\b.*\bgrant', S_text)
    A_defendPartial = A_defendPartial1 or A_defendPartial2
    A_defendGrant = re.search(r'\bdefend\b.*\bmotion\b.*\bgrant', S_text)
    A_defenddeni = re.search(r'\bdefend\b.*\bmotion\b.*\bdeni', S_text)
    A_PlaintiffGrant = re.search(r'\bplaintiff\b.*\bmotion\b.*\bgrant', S_text)
    A_PlaintiffDeni = re.search(r'\bplaintiff\b.*\bmotion\b.*\bdeni', S_text)
    

#####Combinbing all patterns    
    A = [A_Part_for_both, A_Plaintiffgrant_defendPartial, A_Defendgrant_PlaintiffPartial,
         A_Plaintiffdeni_defendPartial, A_defendfdeni_PlaintiffPartial, A_PlaintiffPartial_defendDeni,
         A_PlaintiffPartial_defendGrant, A_DefendfPartial_PlaintiffDeni, A_DefendPartial_PlaintiffGrant, 
         A_PlaintiffGrant_defendGrant, A_PlaintiffGrant_defendDeni, A_PlaintiffDeni_defendGrant, 
         A_PlaintiffDeni_defendDeni, A_DefendGrant_PlaintiffGrant, A_DefendGrant_PlaintiffDeni, 
         A_DefendDeni_PlaintiffGrant, A_DefendDeni_PlaintiffDeni, A_PlaintiffPartial, A_defendPartial, 
         A_defendGrant, A_defenddeni, A_PlaintiffGrant, A_PlaintiffDeni]
    return A
## Function ends

##Results corresponding every feature
Result = ['11A_Part_for_both','12A_Plaintiffgrant_defendPartial', '13A_Defendgrant_PlaintiffPartial', 
          '14A_Plaintiffdeni_defendPartial', '15A_defendfdeni_PlaintiffPartial',
          '16A_PlaintiffPartial_defendDeni', '17A_PlaintiffPartial_defendGrant', 
          '18A_DefendfPartial_PlaintiffDeni', '19A_DefendPartial_PlaintiffGrant', 
          '20A_PlaintiffGrant_defendGrant', '21A_PlaintiffGrant_defendDeni',
          '22A_PlaintiffDeni_defendGrant', '23A_PlaintiffDeni_defendDeni', 
          '24A_DefendGrant_PlaintiffGrant', '25A_DefendGrant_PlaintiffDeni', 
          '26A_DefendDeni_PlaintiffGrant', '27A_DefendDeni_PlaintiffDeni', '28A_PlaintiffPartial', 
          '29A_defendPartial', '30A_DefendGrant', '31A_DefendDeni', 
          '32A_plaintiffGrant', '33A_plaintiffdeni']
###########################################################################

#What if there is no name?
#Motion without naming defend plaintiff
def text1(S_text):
    A_17 = re.search(r'\bmotion\b.*\bdeni\b .* \bgrant', S_text)
    A_17_1 = re.search(r'\bmotion\b.*\bgrant\b .* \bdeni', S_text)
    A_17_2 = re.search(r'\bmotion\b.*\bgrant', S_text)
    A_17_3 = re.search(r'\bmotion\b.*\bdeni', S_text)
    A_Win = [ A_17, A_17_1, A_17_2, A_17_3]
    

#Who moved for summary judge
    A_18 = re.search(r'\bplaintiff\b.*(\bjudgment|\bmotion)', S_text)
    A_19 = re.search(r'\bdefend\b.*(\bjudgment|\bmotion)', S_text)
    A_20 = re.search(r'\bparti\b.*(\bjudgment|\bmotion)', S_text)
    A_Who = [ A_18, A_19, A_20]
    return A_Win, A_Who 
#Function ends
Win = ["Partial", "Partial", "Grant", "Deni"]
Who = ["Plaintiff", "Defend", "Both"]


# Iterating on files

In [75]:
df1 = pd.DataFrame( columns = ["FileName", "Result", "WonStatus", "WhoFiled"])
n = 0
for file in os.listdir("D://Court Misclassification//Known misclass opinions//Predicting motion judgement//Harry text file"):
    d1 = open("D:\Court Misclassification\Known misclass opinions\Predicting motion judgement\Harry text file\%s"%file,'r')
    #cleaning the file:
    d2 = d1.read().replace("Footnotes", "abcdef").lower().replace("'", "").replace("employer", "defendent").replace("company", "defendent").replace(",", "").replace(";", "")
    t1 = [ps.stem(y.decode("utf8","ignore")) for y in d2.split() if not y in stopwords.words("english")]
    
    i = 0
    t_x = []
    if "abcdef" in t1:
        while t1[i] != "abcdef":
            t_x = t_x + [t1[i]]
            i = i +1
    else:
        t_x = t1
    #Taking top page as results are likely to find in first page
    t1_500 = t_x[:350] + t_x[-350:]
    ls = " ".join(t1_500).replace("move", "motion")
    texts = ls.split(". ")
    t = []
    WonStatus = "None"
    WHoFiled = "None"
    #This is to get the top patterns (MIN)- according to the heirachy in above code
    for S_text in texts:
        A = text(S_text)
        v  = min([ i for i in range(0, len(A)) if not A[i] is None] + [60])
        #print v
        if v != 60:
            t = t + [Result[v]]
    Result_1 = min(t + ["60Notabeltofind"])

    #If None of the combination picked up- relaxed the criteria 
    if len(t) == 0:
        for S_text in texts:
            A = text1(S_text)[0]
            for i in range(0, len(A)):
                #print i, S_text, A[i]
                if A[i] is not None:
                    WonStatus = Win[i]
                    break
    if len(t) == 0:
        for S_text in texts:
            B = text1(S_text)[1]
            for i in range(0, len(B)):
                if B[i] is not None:
                    WHoFiled = Who[i]
                    break
    #Saving results data frame 
    FileName = file
    print FileName, Result_1, WonStatus, WHoFiled
    df1.loc[n] = [FileName, Result_1, WonStatus, WHoFiled]
    n = n+1

C:\anaconda\lib\site-packages\ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


Abney v General Electric Co.pdf.txt 60Notabeltofind Deni Plaintiff
Ahmad v Yellow Cab Co of New London and Groton Inc.pdf.txt 30A_DefendGrant None None
Alba v Housing Authority of City of Pittston.pdf.txt 18A_DefendfPartial_PlaintiffDeni None None
Alberty-Velez v Corporacion de Puerto Rico Para La Difusion Publica.pdf.txt 60Notabeltofind None None
Alexander v Avera St Lukes Hosp.pdf.txt 60Notabeltofind None None
Almutairi v International Broadcasting Bureau.pdf.txt 60Notabeltofind Partial Defend
Altman v Sterling Caterers Inc.pdf.txt 13A_Defendgrant_PlaintiffPartial None None
Anyan v New York Life Ins Co.pdf.txt 30A_DefendGrant None None
Art And Drama Therapy Institute Inc v District of Columbia.pdf.txt 30A_DefendGrant None None
Ashkenazi v South Broward Hosp Dist.pdf.txt 60Notabeltofind None Both
Atkins v Computer Sciences Corp.pdf.txt 60Notabeltofind Grant Defend
Barlow v CR England Inc.pdf.txt 60Notabeltofind Grant Defend
Barrows v Teamsters Joint Local 69.pdf.txt 29A_defendPartial 

Penson v Intermodial Research Inc.pdf.txt 60Notabeltofind Grant Defend
Perdomo v Ask 4 Realty And Management Inc.pdf.txt 60Notabeltofind None Defend
Perez v Super Maid LLC.pdf.txt 60Notabeltofind Grant None
Polanco v UPS Freight Services Inc.pdf.txt 60Notabeltofind Partial Defend
Proa v NRT Mid Atlantic Inc.pdf.txt 30A_DefendGrant None None
Ramos v South Florida Express Bankserv Inc.pdf.txt 30A_DefendGrant None None
Randolph v PowerComm Const Inc.pdf.txt 32A_plaintiffGrant None None
Rodriguez-Vega v Policlinica la Familia de Toa Alta Inc.pdf.txt 30A_DefendGrant None None
Rumpke v Rumpke Container Service Inc.pdf.txt 60Notabeltofind Partial Defend
Rush v Watkins Motor Lines Inc.pdf.txt 60Notabeltofind None None
Russell v BSN Medical Inc.pdf.txt 60Notabeltofind Partial Defend
Salamon v Our Lady of Victory Hosp.pdf.txt 60Notabeltofind Deni Defend
Saleem v Corporate Transp Group Ltd.pdf.txt 60Notabeltofind Partial Defend
Saleem v Corporate Transportation Group Ltd.pdf.txt 60Notabeltofind N

In [76]:
df1.to_excel("D://Court Misclassification//Known misclass opinions//Predicting motion judgement//Re_pattern_prediction.xlsx")